Построить информативный дашборд в Redash, объединяющий не менее четырех графиков.

Дашборд для отдела продаж онлайн магазина:

Отображение основных KPI для понимания текущего состояния бизнеса.
Анализ динамики ключевых метрик по странам.
Графики выручки, разбивки по странам, уникальных пользователей, заказов, среднего чека, топ товаров и других метрик.
Данные: ClickHouse → default.retail
Требования к дашборду:

Общая идея дашборда должна быть четко выражена.
Минимум четыре графика, каждый отвечающий на конкретный вопрос.
Графики должны быть наглядны, с четкими названиями и подписями осей.

### Ccылка на дашборд: http://redash.lab.karpov.courses/public/dashboards/UGXuOCAVjvYpfR0tv1u6h87JAGWClcx2FdvA3Iph?org_slug=default

---
Топ 5 стран по выручке без UK (https://redash.lab.karpov.courses/queries/36141/source#73885):

SELECT

    Country,
    
    SUM(Quantity * UnitPrice) AS Total_revenues
    
FROM

    default.retail
    
WHERE 

    Country != 'United Kingdom'
    
GROUP BY

    Country
    
ORDER BY 

    Total_revenues DESC
    
LIMIT 5

Отчет позволяет идентифицировать страны, которые приносят наибольшую выручку, без UK. Это поможет сфокусироваться на этих рынках. 
Нидерланды, Ирландия, Германия, Франция, Австралия являются одним из основных рынков сбыта бизнеса.

---
Динамика выручки и количества заказов по месяцам (https://redash.lab.karpov.courses/queries/36143/source#73890):

SELECT

    Country,
    
    toStartOfMonth(InvoiceDate) AS Data,
    
    SUM(Quantity * UnitPrice) AS Total_revenues
    
FROM

    default.retail
    
WHERE 

    Country IN ('Netherlands', 'EIRE', 'Germany', 'France', 'Australia')
    
GROUP BY

    Country,
    
    Data
    
ORDER BY 

    Total_revenues DESC

График показывает, как меняется выручка в течение года у "пятерки" стран. Найденные сезонные пики могут использоваться для дальнейшего планирования. Например, для грамотного распределения товаров по складам данных стран.
Рассмотрим на примере Нидерландов, график достаточно неустойчив, постоянные скачки говорят о сезонности продаж в стране, разница между 01-07 и 01-08 настолько велика, что выручка моментом взлетела в топ с последнего места.  

---
Топ 5 товаров по количеству продаж в каждой стране (https://redash.lab.karpov.courses/queries/36144/source#73892):

SELECT

    Country, 
    
    Description, 
    
    Total_revenues
    
    FROM (
        
        SELECT
        
            Country,
        
            Description,
        
            SUM(Quantity * UnitPrice) AS Total_revenues,
        
            DENSE_RANK()
        
            OVER(
                
                PARTITION BY Country --раздел в окне по стране
                
                ORDER BY 
                
                    Total_revenues DESC
                
                ) AS qwerty
        
        FROM
        
            default.retail
        
        WHERE 
        
            Country IN ('Netherlands', 'EIRE', 'Germany', 'France', 'Australia')
        
        GROUP BY
        
            Country,
        
            Description
        
    )
    
WHERE qwerty <= 5

Этот график помогает идентифицировать наиболее популярные товары в каждой стране. Вытекающее из пункта 2), на какие товары стоит обратить внимание во время подъема активности покупателей. 
Возьмем все те же Нидерланды, топ пять товаров относительно равный ~22%, но позиция ROUND SNACK BOXES SET OF 4 FRUITS выделяется меньшим спросом.

---
Сравнение среднего чека между странами (https://redash.lab.karpov.courses/queries/36146/source#73896):

SELECT

    Country,
    
    AVG(Quantity * UnitPrice) AS AVG_revenue
    
FROM

    default.retail
    
WHERE 

    Country IN ('Netherlands', 'EIRE', 'Germany', 'France', 'Australia')
    
GROUP BY

    Country
    
ORDER BY 

    AVG_revenue DESC

График сравнивает средние чеки между странами, что может помочь выявить различия в покупательском поведении. Это важно для грамотного ценообразования.
На данном графике явными лидерами оказались Нидерланды и Австралия, это говорит о том, что покупательная способность пользователей там выше, можно попробовать слегка завысить цены. А вот во Франции, Германии и Ирландии стоит наоборот понизить.

---
Топ 25 популярных товаров из изучаемых стран (https://redash.lab.karpov.courses/queries/36149/source#73903):


SELECT

    Description,
    
    SUM(Quantity) AS popular_products
    
FROM

    default.retail
    
WHERE 

    Country IN ('Netherlands', 'EIRE', 'Germany', 'France', 'Australia')
    
GROUP BY

    Description 
    
ORDER BY 

    popular_products DESC
    
LIMIT 25

Данный график отображает 25 популярных товаров во всех изучаемых странах. Это может помочь определить товары, которые являются хитами продаж.

Общий вывод:
    
Анализ этих графиков показывает, какие страны и товары способствуют выручке магазина (за исключением UK, вне конкуренции), как изменяется динамика продаж в течение времени и каковы различия в выборе товара покупателями между странами. Это может стать основой для увеличения эффективности бизнеса.